In [1]:
import tables as tb

import pandas as pd
import numpy as np
import gzip
import json
import os

from nltk import ngrams
from collections import Counter

import preprocessing
from review_embeddings import get_embedding

[nltk_data] Downloading package punkt to /home/tianyiyang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/tianyiyang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/tianyiyang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


torch.Size([1, 768])


In [2]:
DATASET_DIR = "/mnt/nfs/scratch1/neerajsharma/amazon_data/"
META_PREFIX = "meta_"
REVIEW_PREFIX = "review_"
HDF5_DATASET = "dataset.h5"
DATASET_NAME = {
    'phone': 'Cell_Phones_and_Accessories.json.gz',
    'beauty': 'All_Beauty.json.gz',                     # haven't downloaded yet
    'food': 'Grocery_and_Gourmet_Food.json.gz'
}

In [17]:
data_name = 'food'
df = preprocessing.getAmazonData('food', 1000000)
# preprocessing.save_review_dataset(data_name=data_name, dataset=df, dir='./')

In [8]:
#Method for fast conversion from a pytables array to a numpy array
def fast_conversion(hdfarray):
	a=np.empty(shape=hdfarray.shape,dtype=hdfarray.dtype)
	a[:]=hdfarray[:]
	return a

In [9]:
# Read Review Data
data_name = 'food'
with tb.open_file(os.path.join(DATASET_DIR, HDF5_DATASET), 'r') as h5f:
    print(h5f)
    print()
    table = h5f.root[data_name]['Review']
    for i, row in enumerate(table.iterrows()):
        if i == 5:
            break
        print(f"reviewerID: {row['reviewerID'].decode('utf-8')}")
        # print(f"summary: type-{type(row['summary'])}, shape-{row['summary'].shape}")
        print(f"reviewText: type-{type(row['reviewText'])}, shape-{row['reviewText'].shape}\n")
    array = fast_conversion(h5f.root[data_name]['Interactions'])
    print(array)


/mnt/nfs/scratch1/neerajsharma/amazon_data/dataset.h5 (File) ''
Last modif.: 'Fri Mar 19 18:45:07 2021'
Object Tree: 
/ (RootGroup) ''
/food (Group) ''
/food/Interactions (CArray(157277, 6903), shuffle, zlib(5)) ''
/food/Review (Table(157277,)) '{data_name}: Review'


reviewerID: A0004478EF5NFPHLGCWG
reviewText: type-<class 'numpy.ndarray'>, shape-(1, 128)

reviewerID: A00408825PVJW7GFLEGU
reviewText: type-<class 'numpy.ndarray'>, shape-(1, 128)

reviewerID: A0096681Y127OL1H8W3U
reviewText: type-<class 'numpy.ndarray'>, shape-(1, 128)

reviewerID: A014338896MBOX5BLUOF
reviewText: type-<class 'numpy.ndarray'>, shape-(1, 128)

reviewerID: A0172362OTLRVK4CSFHQ
reviewText: type-<class 'numpy.ndarray'>, shape-(1, 128)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
df = df[df['asin'].map(df['asin'].value_counts()) >= 5]
df = df[df['reviewerID'].map(df['reviewerID'].value_counts()) >= 3]
df = df[df['reviewText'].notnull() | (df['reviewText'] == "  ") | (df['reviewText'] == " ") ]

In [20]:
df.asin.nunique()

8601